# ELU 502 Deep learning -- Lab session 4
Pierre-Henri Conze, François Rousseau - session : 1h20

## Objective of this lab session: perform classification on MNIST dataset using convolutional neural networks

In lab session 3, MNIST classification has been performed relying on multi-layer perceptron (MLP). The obtained accuracy was 92% with a simple softmax regressor and 98% with MLP which can be further improved! Let us jump from this simple model to something moderately more sophisticated, namely convolutional neural networks.

For recall, MNIST is a computer vision dataset which consists of handwritten digit images with associated label. Each image in MNIST has a corresponding label, a number between 0 and 9 representing the digit drawn in the image.

### Part 1 - Data management and visualization

First, let us download and read the MNIST data.

In [ ]:
%matplotlib inline
import keras
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
print('x_train shape:', x_train.shape, 'x_test shape:', x_test.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print(x_test[0,:,:].shape, 'image size')

The MNIST data is split into two parts: 60000 data points of training data, 10000 points of test data. Each image is 28 pixels by 28 pixels. 
Let us visualize some of the data.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
plt.figure()
for i in range(10):
    plt.subplot(2, 5, i + 1)
    plt.axis('off')
    index = np.where(y_train == i)[0][0]
    plt.imshow(x_train[index,:,:],cmap=plt.cm.gray_r)
    plt.title('Training: %i' % y_train[index])
plt.show()

For further processing using keras functions, MNIST data must stacked in a 4D tensor with shape (samples, rows, cols, channels). In our case, channels=1 since we are working with greyscale images. As in lab session 3, pixel intensities need to rescaled between 0 and 1. Moreovre, labels can be described as one-hot vectors using the to_categorical() keras function.

In [ ]:
img_rows, img_cols = 28, 28
num_classes = 10
# build 4D tensors
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
# data normalization
x_train = x_train.astype('float32')/255.
x_test = x_test.astype('float32')/255.
# convert class vectors to binary class matrices
z_train = keras.utils.to_categorical(y_train, num_classes)
z_test = keras.utils.to_categorical(y_test, num_classes)

### Part 2 - Building a convolutional neural network with keras

The sequential Keras model is a linear stack of layers. You can create a sequential model by passing a list of layer instances to the constructor. Among the layer instances, you can use:
  - convolutional layer using Conv2D (https://keras.io/layers/convolutional/#conv2d)
  - max-pooling layer using MaxPooling2D (https://keras.io/layers/pooling/#maxpooling2d)
  - Dropout (https://keras.io/layers/core/#dropout)
  - regular densely-connected layer using Dense (https://keras.io/layers/core/#dense)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Once keras modules have been imported, we can simply add layers using the .add() method.

In addition, the model needs to know what input shape it should expect. For this reason, the first layer in a Sequential model (and only the first, because following layers can do automatic shape inference) needs to receive information about its input shape:

In [ ]:
input_shape = (img_rows, img_cols, 1)

#### 1) Create a convolutional neural network following the architecture given below:
 - convolutional layer using 32 3x3 filters with stride 1 and "ReLU" activation
 - convolutional layer using 64 3x3 filters with stride 1 and "ReLU" activation
 - max pooling with vertical, horizontal downscale of 2
 - flatten layer (https://keras.io/layers/core/#flatten) to flatten the input array
 - dense layer with 128 units
 - dense layer with "num_classes=10" units

In [ ]:
# model = Sequential()
# model.add(...) # to do

Before training a model, you need to configure the learning process, which is done through the compile() method. It receives three arguments:

 - an optimizer: this could be the string identifier of an existing optimizer (such as rmsprop or adagrad), or an instance of the Optimizer class.
 - A loss function: this is the objective that the model will try to minimize. It can be the string identifier of an existing loss function (such as categorical_crossentropy or mse), or an objective function.
 - A list of metrics to evaluate results. A metric could be the string identifier of an existing metric or a custom metric function.

#### 2) Define the loss, the optimizer and the metrics. Then, compile your model. You can find some help from https://keras.io/models/sequential/#compile. 

In [ ]:
# model.compile(...) # to do

#### 3) Describe input/output sizes of each layer. Confirm your analysis by using model.summary()

In [ ]:
# to do

#### 4) Keras models are trained on Numpy arrays of input data and labels. For training a model, we can use the fit() function (https://keras.io/models/sequential/#fit). Run the training using a batch size of 128 and 12 epochs. Test data will be used as validatation data.

In [ ]:
# to do

### Part 3 - Evaluation

The model.fit() method returns an History callback, which has a history attribute containing the lists of successive losses and other metrics.

#### 5) Display the loss and the accuracy across training 

In [ ]:
# to do

In [ ]:
# to do

#### 6) Evaluate the global test loss and accuracy using the evaluate() method (https://keras.io/models/sequential/#evaluate).

In [ ]:
# to do

#### 7) Visualize some wrongly predicted data

In [ ]:
# to do

Up to now, we have learned how to quickly and easily build, train, and evaluate a convolutional neural network using Keras. The final test set accuracy on MNIST dataset is approximately 99% which is better than the results obtained with MLP.

### Part 4 - Robustness to noise

#### 8) Evaluate the network predictions on noisy data with two different noise factors

In [ ]:
noise_factor = 0.2
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)
plt.figure()
for i in range(10):
    plt.subplot(2, 5, i + 1)
    plt.axis('off')
    index = np.where(y_test == i)[0][0]
    plt.imshow(x_test_noisy[index,:,:,0], cmap=plt.cm.gray_r)
    plt.title('%i' % y_test[index])
plt.show()
score = model.evaluate(x_test_noisy, z_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


In [ ]:
noise_factor = 0.4
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)
plt.figure()
for i in range(10):
    plt.subplot(2, 5, i + 1)
    plt.axis('off')
    index = np.where(y_test == i)[0][0]
    plt.imshow(x_test_noisy[index,:,:,0], cmap=plt.cm.gray_r)
    plt.title('%i' % y_test[index])
plt.show()
score = model.evaluate(x_test_noisy, z_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])



With strong noise, the accuracy decreases from 99% to 80% since training samples are not enough representative. Let us train the model directly on noisy data samples to see the performance.

#### 9) Train and evaluate the same CNN architecture on noisy data with 0.4 as noise factor. 

In [ ]:
# x_train_noisy = ...
# model_noise = Sequential()
# to do

#### 10) What is the expected result when using Dropout layers? Evaluate the network without Dropout layers and conclude based on global test accuracy 

Dropout consists in randomly setting a fraction rate of input units to 0 at each update during training time, which helps prevent overfitting.

In [ ]:
# model_noise2 = Sequential()
# to do

For this small convolutional network, performance is actually nearly identical with and without dropout. Here, when training and predict on noisy data, we jump from 97,8% (without dropout) to 98,0% (with dropout). Dropout is often very effective at reducing overfitting, but it is most useful when training very large neural networks.

### Part 5 - Towards more deeper networks

In this last part, we come back to the original dataset, i.e. without additional noise.

#### 11) Implement a deeper convolutional neural network by adding two convolutional  layers and one max pooling layer before the first Dropout. What is the performance gain?

In [ ]:
model2 = Sequential()
# to do

This slightly more deeper network reaches around 99,35% in terms of accuracy (instead of 99% as in question 6). 

#### 12) Study the confusion matrix:

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools

z_pred = model2.predict(x_test)
y_pred = np.argmax(z_pred,axis=1)

class_names= ['0','1','2','3','4','5','6','7','8','9']

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    #print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

To conclude, an overview of results obtained with different methodologies can be found here: https://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.